In [1]:
using TensorFlow
using Distributions
using MLDataUtils
using SwiftObjectStores
using Iterators
using ColoringNames
using MappedArrays
using MLLabelUtils
using StaticArrays
using Base.Test
using ProgressMeter
using IJulia
using Memoize
import DataFramesMeta.@with

In [91]:
stat(serv, "color")["items"]

8-element Array{Any,1}:
 ("Account","AUTH_eca532d8c1e84913bd21d6d22b30f985")
 ("Container","color")                              
 ("Objects","3")                                    
 ("Bytes","84645914")                               
 ("Read ACL","")                                    
 ("Write ACL","")                                   
 ("Sync To","")                                     
 ("Sync Key","")                                    

In [93]:
const serv=SwiftService()
const train_raw = get_file(serv, "color", "monroe/train.csv") do fh
    readdlm(fh,'\t')
end

const valid_raw = get_file(serv, "color", "monroe/dev.csv") do fh
    readdlm(fh,'\t')
end

108545×2 Array{Any,2}:
 "acid green"  "(0.328092243187,0.665271966527,0.937254901961)" 
 "acid green"  "(0.241007194245,0.712820512821,0.764705882353)" 
 "acid green"  "(0.274706867672,0.780392156863,1.0)"            
 "acid green"  "(0.35346215781,0.896103896104,0.905882352941)"  
 "acid green"  "(0.22385620915,0.846473029046,0.945098039216)"  
 "acid green"  "(0.219101123596,0.700787401575,0.996078431373)" 
 "acid green"  "(0.205459770115,0.954732510288,0.952941176471)" 
 "acid green"  "(0.184782608696,0.987124463519,0.913725490196)" 
 "acid green"  "(0.204225352113,0.883817427386,0.945098039216)" 
 "acid green"  "(0.253875968992,0.830917874396,0.811764705882)" 
 "acid green"  "(0.299065420561,0.968325791855,0.866666666667)" 
 "acid green"  "(0.298711755233,0.93665158371,0.866666666667)"  
 "acid green"  "(0.287596899225,0.968468468468,0.870588235294)" 
 ⋮                                                              
 "yuck"        "(0.142694063927,0.679069767442,0.843137254902)" 
 "

In [3]:
const tokenize = morpheme_tokenizer("../data/replacement_rules.csv")
@memoize demarcate(tokens, starter="<S>", ender="</S>") = [starter; tokens; ender]

@test demarcate(["a", "b", "c"]) == ["<S>", "a", "b", "c", "</S>"]

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in symbol(::String, ::Vararg{String,N

In [4]:

const od =(MLDataUtils.ObsDim.First(), MLDataUtils.ObsDim.Last())


In [5]:
function prepare_data(raw, encoding_=nothing)
    const labels = convert(Vector{String}, raw[:,1]);
    const hsv_data = convert(Matrix{Float64}, raw[:,2:end]);
    const tokenized_labels = demarcate.(tokenize.(labels))
    
    if encoding_===nothing
        const all_tokens = reduce(union, tokenized_labels)
        const encoding = labelenc(all_tokens)        
    end
    const label_inds = map(toks->label2ind.(toks, Scalar(encoding)), tokenized_labels)
end


1523108-element Array{Array{String,1},1}:
 String["<S>","acid","green","</S>"]
 String["<S>","acid","green","</S>"]
 String["<S>","acid","green","</S>"]
 String["<S>","acid","green","</S>"]
 String["<S>","acid","green","</S>"]
 String["<S>","acid","green","</S>"]
 String["<S>","acid","green","</S>"]
 String["<S>","acid","green","</S>"]
 String["<S>","acid","green","</S>"]
 String["<S>","acid","green","</S>"]
 String["<S>","acid","green","</S>"]
 String["<S>","acid","green","</S>"]
 String["<S>","acid","green","</S>"]
 ⋮                                  
 String["<S>","yuck","</S>"]        
 String["<S>","yuck","</S>"]        
 String["<S>","yuck","</S>"]        
 String["<S>","yuck","</S>"]        
 String["<S>","yuck","</S>"]        
 String["<S>","yuck","</S>"]        
 String["<S>","yuck","</S>"]        
 String["<S>","yuck","</S>"]        
 String["<S>","yuck","</S>"]        
 String["<S>","yuck","</S>"]        
 String["<S>","yuck","</S>"]        
 String["<S>","yuck","</S>"]     

Test Summary: | Pass  

In [9]:
const padded_labels = rpad_to_matrix(label_inds)

6×1523108 Array{Int64,2}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …    1    1    1    1    1    1    1
 2  2  2  2  2  2  2  2  2  2  2  2  2     313  313  313  313  313  313  313
 3  3  3  3  3  3  3  3  3  3  3  3  3       4    4    4    4    4    4    4
 4  4  4  4  4  4  4  4  4  4  4  4  4       0    0    0    0    0    0    0
 0  0  0  0  0  0  0  0  0  0  0  0  0       0    0    0    0    0    0    0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …    0    0    0    0    0    0    0

In [10]:
get_mask(V, dtype=Float32)=cast(cast(V, Bool),dtype)
apply_mask(V, mask) = V.*tile(expand_dims(mask, 1), [1, get_shape(V,2)])

@testset "masks" begin
    mask_sess = Session(Graph())
    a_val = (
    [2.  5  8  9  2
     3.  6  4  4  9
     4.  7   3  2  4
     1.  4  2  4  4]
    )
    A=constant(a_val)
    M = constant([1, 0, 2, 4])
    run(mask_sess, initialize_all_variables())
    masked = run(mask_sess, apply_mask(A, get_mask(M, Float64)))
    @test all(masked[2,:] .== 0)
    @test all(masked[1,:] .== a_val[1,:])
    @test all(masked[3:4,:] .== a_val[3:4,:])
end;

Test Summary: | Pass  Total
  masks       |    3      3


W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.


Test Summary: | Pass  Total
  names_from  |    3      3


In [13]:
#DEFINITION
function color_to_terms_network(n_classes, n_steps;
        batch_size = 128,
        learning_rate = 0.05,
        hidden_layer_size = 256,
        embedding_dim = 16
    )
    n_input = 3 # HSV
    
        
    ###################################
    sess = Session(Graph())
    tensor_vars = @names_from begin
        X_hsv = placeholder(Float32, shape=[batch_size, n_input]; name="X_HSVs")
        Term_obs_s = placeholder(Int32, shape=[n_steps+1, batch_size]; name="Term_obs_s")

        EmbeddingTable = get_variable("TokenEmbeddings",  [n_classes, embedding_dim], Float32; initializer=Normal(0, .1))

        #Mangle Terms into shape
        Term_obs_s_out = slice(Term_obs_s, [1,0], [-1,-1]) #Don't want first input "<S>" #0based
        Term_obs_s_ins = unpack(Term_obs_s+1)[1:end-1]#Don't want last input "</S>" (or padding character often but we will handle that seperately)
        Term_obs_s_ins_r = reshape.(Term_obs_s_ins, Scalar(batch_size))
        Tes = gather.(Scalar(EmbeddingTable), Term_obs_s_ins_r)

        TT = reshape(Term_obs_s_out, [n_steps*batch_size]; name="Stack_Term_obs_s")
        Term_obs_onehots = one_hot(TT, n_classes)


        #Mangle colots into shape
        X_h, X_s, X_v = unpack(X_hsv; axis=2)
        X_hr = X_h.*2π
        X_col = pack((sin(X_hr), cos(X_hr), X_s-0.5, X_v-0.5)) #Smooth hue by breaking into cos and sin, and zero mean everything else
        X_col = reshape(X_col, [batch_size, 4]; name = "X_col") #Force Shape Inference
        Xs = [concat(1, (T, X_col); name="Xs$ii") for (ii,T) in enumerate(Tes)]#Pair color input at each step with previous term #0base

        Hs, states = nn.rnn(nn.rnn_cell.LSTMCell(hidden_layer_size), Xs; dtype=Float32)#, sequence_length=n_steps);
        W1 = get_variable("weights1", [hidden_layer_size, n_classes], Float32;  initializer=Normal(0, .1))
        B1 = get_variable("bias1", [n_classes], Float32;  initializer=Normal(0, .1))
        Ls =  [H*W1+B1 for H in Hs]


        LL = concat(0, Ls; name="Stack_Logits") #Stack #0base
        LL = reshape(LL, [batch_size*n_steps, n_classes]) #Force Shape Inference
        LL_masked = apply_mask(LL, get_mask(TT))

        Term_preds_onehots = nn.softmax(LL_masked; name="Term_preds_onehots")
        Term_preds_onehots_log = nn.log_softmax(LL_masked; name="Term_preds_onehots_log")
        Term_preds_s = reshape(indmax(Term_preds_onehots, 2)+1, [n_steps, batch_size]) #TODO: this messes up zero entries, not that it matters

        costs  = reduce_sum(Term_obs_onehots.*Term_preds_onehots_log, reduction_indices=[1])
        #costs = nn.sparse_softmax_cross_entropy_with_logits(LL, TT+1)
        cost = reduce_mean(-costs) #cross entropy

        optimizer = train.minimize(train.AdamOptimizer(learning_rate), cost)
    end
    ########## GET it running
    
    run(sess, initialize_all_variables())
    
    return sess, tensor_vars 
end

color_to_terms_network (generic function with 1 method)

In [63]:
const batch_size = 128

sess, t = color_to_terms_network(nlabel(encoding)+1, size(padded_labels,1)-1;
        learning_rate = 0.05,
        hidden_layer_size = 256,
        embedding_dim = 16,
        batch_size=batch_size
    )            

****

(Session(Ptr{Void} @0x00007f2d15d18510),Dict{Symbol,Any}(Pair{Symbol,Any}(:X_col,<Tensor X_col_3:1 shape=(128, 4) dtype=Float32>),Pair{Symbol,Any}(:Term_obs_onehots,<Tensor OneHot_3:1 shape=(640, 314) dtype=Float32>),Pair{Symbol,Any}(:Term_obs_s_ins,TensorFlow.Tensor[<Tensor Unpack_6:1 shape=unknown dtype=Int32>,<Tensor Unpack_6:2 shape=unknown dtype=Int32>,<Tensor Unpack_6:3 shape=unknown dtype=Int32>,<Tensor Unpack_6:4 shape=unknown dtype=Int32>,<Tensor Unpack_6:5 shape=unknown dtype=Int32>]),Pair{Symbol,Any}(:Tes,TensorFlow.Tensor[<Tensor Gather_16:1 shape=(128, 16) dtype=Float32>,<Tensor Gather_17:1 shape=(128, 16) dtype=Float32>,<Tensor Gather_18:1 shape=(128, 16) dtype=Float32>,<Tensor Gather_19:1 shape=(128, 16) dtype=Float32>,<Tensor Gather_20:1 shape=(128, 16) dtype=Float32>]),Pair{Symbol,Any}(:Term_preds_onehots,<Tensor Term_preds_onehots_3:1 shape=unknown dtype=Float32>),Pair{Symbol,Any}(:Xs,TensorFlow.Tensor[<Tensor Xs1_3:1 shape=(128, 20) dtype=Float32>,<Tensor Xs2_3:1 sha

In [64]:
function train_from_terms!(sess, t::Associative{Symbol})
    batchs = eachbatch(shuffleobs((hsv_data, padded_labels); obsdim=od); size=batch_size, obsdim=od)
    local cost_o
    @showprogress for (hsv,terms) in batchs
        Term_preds_s_o, TT_s_o, cost_o, optimizer_o = run(sess,
            [t[:Term_preds_s], t[:TT], t[:cost], t[:optimizer]], 
        Dict(t[:X_hsv]=>hsv, t[:Term_obs_s]=>terms))
    end
    @show cost_o
end
#obs,pred, oo, pp = run(sess, [Term_obs_onehots, Term_preds_onehots, Term_obs_s_out, Term_preds_s], Dict(X_hsv=>hsv_data, Term_obs_s=>padded_labels))
train_from_terms!(sess, t)

INFO: The specified values for size and/or count will result in 36 unused data points


Progress:  83%|██████████████████████████████████       |  ETA: 0:02:34

Excessive output truncated after 524439 bytes.

2.1531072f0

In [85]:
        Term_preds_s_o, TT_s_o, cost_o, optimizer_o = run(sess,
            [t[:Term_preds_s], t[:TT], t[:cost], t[:optimizer]], 
        Dict(t[:X_hsv]=>hsv, t[:Term_obs_s]=>terms))
    end

Session(Ptr{Void} @0x00007f2d15d18510)

In [82]:
TensorFlow.get_tensors([t[:W1]])

1-element Array{Any,1}:
 TensorFlow.Variable(<Operation 'weights1'>,<Operation 'weights1/Assign'>)

In [79]:
TensorFlow.num_outputs( TensorFlow.get_op(t[:W1]))

1

In [25]:
    if test_it 
        @assert isa(run(sess, cost, Dict(X_hsv=>hsv_data, Term_obs_s=>padded_labels)), Number )
        ## MASK TEST######################
        ll, ll2 = run(sess, [ LL_masked, 
            reshape(tile(expand_dims(get_mask(Term_obs_s_out),2),[1,1,n_classes]).*concat(0, expand_dims.(Ls, Scalar(0))), [batch_size*n_steps, n_classes])
        ], Dict(X_hsv=>hsv_data, Term_obs_s=>padded_labels))
        @assert ll ≈ ll2
    end

LoadError: UndefVarError: test_it not defined

In [ ]:
ind2label.(ans, Scalar(encoding))

In [ ]:
##MASK test code
#TODO 
if test_it 
        @assert isa(run(sess, cost, Dict(X_hsv=>hsv_data, Term_obs_s=>padded_labels)), Number )
        ## MASK TEST######################
        ll, ll2 = run(sess, [ LL_masked, 
            reshape(tile(expand_dims(get_mask(Term_obs_s_out),2),[1,1,8]).*concat(0, expand_dims.(Ls, Scalar(0))), [batch_size*n_steps, n_classes])
        ], Dict(X_hsv=>hsv_data, Term_obs_s=>padded_labels))
        @assert ll ≈ ll2
    end

In [ ]:
rank(X_col)

In [ ]:
function dict2type(vars::Associative{Symbol})
     
end
"""Take dictionary of named value, and brings each of them into scope with the indicated names"""
macro with_vars(vars, body)
    varsdec = Expr(:block, (Expr(:(=), name, :($(esc(vars))[$(name)]) for (name) in vars)...)
    quote 
        $varsdec
        $esc(body)
    end
end

@testset "bring_into_scope" begin
    vartab = Dict(:x2=>2, :y2=>30)
    @with_vars vartab begin
        @test y2==30
        @test x2==2
    end
    function test_inside_fun()
        vartab
        @eval :(esc(into_scope(Dict(:x4=>2000, :y4=>300))))
        @test y4==300
        @test x4==2000
    end
    test_inside_fun()
    @test y4==30
    @test x4==2
end


